# HW 5 Question 2

# Setup

In [19]:
%matplotlib inline
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import torch
import torch.optim as optim
torch.set_printoptions(edgeitems=2, linewidth=75)

## define model and functions

In [20]:
def model(feature5, feature4, feature3, feature2, feature1, w5, w4, w3, w2, w1, b):
    return feature5 * w5 + feature4 * w4 + feature3 * w3 + feature2 * w2 + feature1 * w1 + b

In [21]:
def loss_function(predicted, actual):
    squared_diffs = (predicted - actual)**2
    return squared_diffs.mean()

In [22]:
rates_to_learn_at = [1/x for x in [10, 100, 1000, 10000, 100000]]
rates_to_learn_at

[0.1, 0.01, 0.001, 0.0001, 1e-05]

## Data import

In [23]:
housing_df = pd.DataFrame(pd.read_csv('Housing.csv'))
housing_df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [24]:
print(f"shape = {np.shape(housing_df)}")

# creates a list of all variables from the column names
feature_list = list( housing_df.columns )

print(f"features are: {feature_list}")

shape = (545, 13)
features are: ['price', 'area', 'bedrooms', 'bathrooms', 'stories', 'mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'parking', 'prefarea', 'furnishingstatus']


In [25]:
# Maps to turn categorys into numbers 
def boolean_map(x):
    return x.map({'yes': 1 , 'no': 0})
def furnish_map(x):
    return x.map({'furnished': 1 , 'semi-furnished': 0.5 , 'unfurnished': 0})

# Extracts the yes and no column names
binary_vars = [*feature_list[5:10], feature_list[11]]
print(f"binary vars = {binary_vars}")

# Extracts the furnishing column names
furnish_vars = [feature_list[12]]
print(f"furnish vars = {furnish_vars}")

# Extracts the column names that are actual values
valued_vars = feature_list.copy()
[valued_vars.remove( item ) for item in binary_vars]
[valued_vars.remove( item ) for item in furnish_vars]
print(f"value vars = {valued_vars}")

binary vars = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']
furnish vars = ['furnishingstatus']
value vars = ['price', 'area', 'bedrooms', 'bathrooms', 'stories', 'parking']


In [26]:
x_df = housing_df.copy()

## scale data
scaler = StandardScaler()
# scaler = MinMaxScaler()

x_df[valued_vars] = scaler.fit_transform(x_df[valued_vars])

## map text values
x_df[binary_vars] = x_df[binary_vars].apply(boolean_map)
x_df[furnish_vars] = x_df[furnish_vars].apply(furnish_map)

## make y_df
y_df = x_df.pop('price')


In [27]:
# input_values = valued_vars.copy()
# input_values.remove('price')


# x_df = x_df[input_values]

In [28]:
x_df.head()

,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,1.046726,1.403419,1.421812,1.378217,1,0,0,0,1,1.517692,1,1.0
1,1.757010,1.403419,5.405809,2.532024,1,0,0,0,1,2.679409,0,1.0
2,2.218232,0.047278,1.421812,0.224410,1,0,1,0,0,1.517692,1,0.5
3,1.083624,1.403419,1.421812,0.224410,1,0,1,0,1,2.679409,1,1.0
4,1.046726,1.403419,-0.570187,0.224410,1,1,1,0,1,1.517692,0,1.0


In [29]:
y_df.head()

0    4.566365
1    4.004484
2    4.004484
3    3.985755
4    3.554979
Name: price, dtype: float64

In [30]:
# Remove unwanted data
for item in [*binary_vars, *furnish_vars] :
    x_df.pop(item)
x_df.head()

,area,bedrooms,bathrooms,stories,parking
0,1.046726,1.403419,1.421812,1.378217,1.517692
1,1.757010,1.403419,5.405809,2.532024,2.679409
2,2.218232,0.047278,1.421812,0.224410,1.517692
3,1.083624,1.403419,1.421812,0.224410,2.679409
4,1.046726,1.403419,-0.570187,0.224410,1.517692


In [36]:
# Convert data frame to tensor

# TODO

## Split into train and validation sets

In [37]:
n_samples = x_df.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

train_indices, val_indices  # <1>

(tensor([541, 372, 214,  73, 133, 242, 258, 147, 102, 539, 211, 198,  58,
         246, 457,  20, 421, 310, 245, 176, 274, 456, 326, 107, 362, 366,
         431, 193, 370, 138, 398, 163, 422, 262, 354, 530, 515, 166,  59,
         286,   8,  91, 281,  96, 110, 357,  42, 532, 405, 201, 340, 484,
          34, 418, 544, 252,  72, 497, 288, 194, 443, 315, 415, 523, 476,
          75, 407, 410,  28, 105, 299, 314, 512, 331, 374, 458, 466, 330,
          10, 414, 172, 234, 227,  23, 140, 155, 373, 273,  46, 433, 283,
         391,  78,  15, 131, 240, 367, 402, 481,  43, 257,  33, 352, 232,
         540, 182, 464, 164, 313, 449,  41, 212, 143, 335, 248, 268,  40,
         531, 167, 432, 339, 424, 106,   4, 355,  11,  63, 325, 197, 479,
         525, 329, 295, 222, 109, 492, 509, 419, 130, 439, 361, 213, 275,
         487, 349, 491, 527,  16,  83,  53,  82,  79, 337, 184,  69, 490,
          12,  90, 397, 381, 296, 323, 230, 519, 108,  36, 543, 244,   5,
         508, 389, 179, 289, 218,  50,

In [38]:
training_feature = data_feature[train_indices]
training_output  = data_output [train_indices]

validation_feature = data_feature[val_indices]
validation_output  = data_output[val_indices]

# training_un   = 0.1 * training_feature
# validation_un = 0.1 * validation_feature

NameError: name 'data_feature' is not defined

## Define the training

In [39]:
def training_loop(n_epochs, optimizer, params, train_t_u, val_t_u,
                  train_t_c, val_t_c):
    for epoch in range(1, n_epochs + 1):
        train_t_p = model(train_t_u, *params)
        train_loss = loss_function(train_t_p, train_t_c)

        # For the valudation step
        with torch.no_grad(): 
            val_t_p = model(val_t_u, *params)
            val_loss = loss_function(val_t_p, val_t_c)
            assert val_loss.requires_grad == False # <2>
            
        optimizer.zero_grad()
        
        train_loss.backward()
        optimizer.step()
        
        if epoch <= 3 or epoch % 500 == 0:
            print(f"Epoch {epoch}, Training loss {train_loss.item():.4f},"
                  f" Validation loss {val_loss.item():.4f}")

# Results

In [42]:
params = torch.tensor([1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-4
optimizer = optim.SGD([params], lr=learning_rate)

training_loop(
    n_epochs = 5000, 
    optimizer = optimizer,
    params = params,
    train_t_u = training_feature, # <1> 
    val_t_u = validation_feature, # <1> 
    train_t_c = training_t_c,
    val_t_c = validation_t_c)

params

NameError: name 'training_feature' is not defined

In [43]:
params = torch.tensor([1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-4
optimizer = optim.Adam([params], lr=learning_rate)

training_loop(
    n_epochs = 5000, 
    optimizer = optimizer,
    params = params,
    train_t_u = training_t_un, # <1> 
    val_t_u = validation_t_un, # <1> 
    train_t_c = training_output,
    val_t_c = validation_output)

params

NameError: name 'training_t_un' is not defined